In [2]:
import pyomo.environ as pyo
s = pyo.SolverFactory("ipopt")
print("Ipopt available:", s.available())

Ipopt available: True


In [3]:
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt

from idaes.core import FlowsheetBlock
from watertap.core.wt_database import Database
from watertap.core.zero_order_properties import WaterParameterBlock
from watertap.unit_models.zero_order import dual_media_filtration_zo

import pyomo.environ as pyo

def get_pyomo_solver():
    # Use Ipopt directly (no Pynumero / WaterTAP wrapper)
    solver = pyo.SolverFactory("ipopt")
    if not solver.available():
        raise RuntimeError("Ipopt solver is not available in this environment.")
    return solver



# ---------------------------------------------------------------------------
# 1. Load and prepare training data
# ---------------------------------------------------------------------------

def load_training_data(csv_path: str) -> pd.DataFrame:
    """
    Load the synthetic WTP/WaterTAP training dataset and check the key columns.
    This matches wtp_watertap_training_dataset.csv.
    """
    df = pd.read_csv(csv_path)

    required_cols = [
        "timestamp",
        "flow_m3_s",
        "flow_mass_H2O_kg_s",
        "flow_mass_TSS_in_kg_s",
        "flow_mass_TSS_out_kg_s",
        "raw_temperature_C",
        "TSS_raw_mgL",
        "TSS_settled_mgL",
    ]

    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in CSV: {missing}")

    return df



# ---------------------------------------------------------------------------
# 2. Calibration of TSS removal fraction using Pyomo
#     (mirrors the pattern in the video ~minute 33)
# ---------------------------------------------------------------------------

def train_tss_removal_fraction_with_pyomo(df: pd.DataFrame) -> float:
    """
    Fit a single removal fraction r_tss such that:
        TSS_out_measured ≈ (1 - r_tss) * TSS_in
    using your synthetic dataset (TSS_raw_mgL, TSS_settled_mgL).
    """

    # Use mg/L directly from CSV
    tss_in = df["TSS_raw_mgL"].values
    tss_out = df["TSS_settled_mgL"].values
    n = len(df)

    m = pyo.ConcreteModel()
    m.T = pyo.RangeSet(0, n - 1)
    m.r_tss = pyo.Var(bounds=(0, 1), initialize=0.8)
    m.eps = pyo.Var(m.T)

    def eps_def_rule(m, t):
        return m.eps[t] == tss_out[t] - (1 - m.r_tss) * tss_in[t]

    m.eps_def = pyo.Constraint(m.T, rule=eps_def_rule)
    m.obj = pyo.Objective(expr=sum(m.eps[t] ** 2 for t in m.T), sense=pyo.minimize)

    solver = get_pyomo_solver()
    solver.solve(m, tee=False)

    r_hat = pyo.value(m.r_tss)
    print(f"Calibrated TSS removal fraction (dataset-wide): {r_hat:.3f}")
    return r_hat



# ---------------------------------------------------------------------------
# 3. Minimal WaterTAP flowsheet: DMF only
# ---------------------------------------------------------------------------

def build_dmf_flowsheet(db: Database, r_tss: float):
    """
    Build a minimal zero-order flowsheet with a single DualMediaFiltrationZO unit
    and set its TSS removal fraction to the calibrated value.
    """
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # Zero-order property package with TSS as the only solute
    m.fs.properties = WaterParameterBlock(solute_list=["tss"])

    # Zero-order dual media filtration unit
    m.db = db
    m.fs.dmf = dual_media_filtration_zo.DualMediaFiltrationZO(
        property_package=m.fs.properties,
        database=m.db,
        process_subtype="default",  # adjust if you have a specific subtype
    )

    # Load default parameters from the WaterTAP database
    m.fs.dmf.load_parameters_from_database()

    # Overwrite the TSS removal fraction with the calibrated value
    # Index (0, "tss") assumes time=0, component="tss" (standard in WT zero-order)
    m.fs.dmf.removal_frac_mass_comp[0, "tss"].fix(r_tss)

    return m


def fix_inlet_state_from_row(m, row):
    """
    Set inlet state for DMF zero-order unit using the CSV row.

    CSV gives:
      - flow_m3_s       [m³/s]
      - TSS_raw_mgL     [mg/L]
    WaterTAP expects:
      - flow_vol        [m³/s]
      - conc_mass_comp  [kg/m³]
    """

    # Read from CSV
    Q_m3_s = float(row["flow_m3_s"])
    tss_mgL = float(row["TSS_raw_mgL"])

    # mg/L -> kg/m³ (1 mg/L = 1e-3 kg/m³)
    tss_kg_m3 = tss_mgL * 1e-3

    u = m.fs.dmf

    # For DMF_0D the state is usually on the "inlet_state" block
    u.inlet_state[0].flow_vol.fix(Q_m3_s)
    u.inlet_state[0].conc_mass_comp["tss"].fix(tss_kg_m3)
    u.inlet_state[0].temperature.fix(298.15)  # K
    u.inlet_state[0].pressure.fix(101325)     # Pa




# ---------------------------------------------------------------------------
# 4. Run WaterTAP simulation for the whole dataset
# ---------------------------------------------------------------------------

def simulate_dataset_with_dmf(m, df: pd.DataFrame) -> pd.DataFrame:
    """
    Loop over all rows in df, fix the inlet state, solve the DMF model,
    and record inlet/outlet flows and TSS concentrations.

    Returns a new dataframe with model outputs.
    """
    solver = get_pyomo_solver()
    results_rows = []

    for i, row in df.iterrows():
        # 1) Set inlet state from CSV
        fix_inlet_state_from_row(m, row)

        # 2) Solve
        solver.solve(m, tee=False)

        # 3) Get properties in/out
        u = m.fs.dmf
        props_in = u.properties_in[0]
        props_out = u.properties_out[0]

        # Volumetric flows [m3/s] – expressions in the property package
        flow_in_m3_s = pyo.value(props_in.flow_vol)
        flow_out_m3_s = pyo.value(props_out.flow_vol)

        # TSS concentrations [kg/m3] -> [mg/L] (1 kg/m3 = 1000 mg/L)
        tss_in_kg_m3 = pyo.value(props_in.conc_mass_comp["tss"])
        tss_out_kg_m3 = pyo.value(props_out.conc_mass_comp["tss"])

        tss_in_mgL_model = tss_in_kg_m3 * 1000.0
        tss_out_mgL_model = tss_out_kg_m3 * 1000.0

        results_rows.append(
            {
                "timestamp": row["timestamp"],
                "flow_in_m3_s_model": flow_in_m3_s,
                "flow_out_m3_s_model": flow_out_m3_s,
                "TSS_in_mgL_model": tss_in_mgL_model,
                "TSS_out_mgL_model": tss_out_mgL_model,
            }
        )

    return pd.DataFrame(results_rows)

# ---------------------------------------------------------------------------
# 5. Visualisation
# ---------------------------------------------------------------------------

def plot_dmf_results(df_data: pd.DataFrame, df_model: pd.DataFrame):
    """
    Compare measured vs modelled flows and TSS
    at the DMF inlet/outlet.
    """

    # Align on timestamp
    df_data = df_data.copy()
    df_data["timestamp"] = pd.to_datetime(df_data["timestamp"])
    df_model = df_model.copy()
    df_model["timestamp"] = pd.to_datetime(df_model["timestamp"])

    df_merged = pd.merge(df_data, df_model, on="timestamp", how="inner")

    t = df_merged["timestamp"]

    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # --- Flows ---
    ax0 = axes[0]
    ax0.plot(t, df_merged["flow_m3_s"], label="Inlet flow (data)")
    ax0.plot(t, df_merged["flow_in_m3_s_model"], linestyle="--", label="Inlet flow (model)")
    ax0.plot(t, df_merged["flow_out_m3_s_model"], linestyle=":", label="Outlet flow (model)")
    ax0.set_ylabel("Flow [m³/s]")
    ax0.legend()
    ax0.grid(True)

    # --- TSS ---
    ax1 = axes[1]
    ax1.plot(t, df_merged["TSS_raw_mgL"], label="TSS inlet (data)")
    ax1.plot(t, df_merged["TSS_settled_mgL"], label="TSS settled (data)")
    ax1.plot(t, df_merged["TSS_in_mgL_model"], linestyle="--", label="TSS inlet (model)")
    ax1.plot(t, df_merged["TSS_out_mgL_model"], linestyle=":", label="TSS outlet (model)")
    ax1.set_ylabel("TSS [mg/L]")
    ax1.set_xlabel("Time")
    ax1.legend()
    ax1.grid(True)

    plt.tight_layout()
    plt.show()


# ---------------------------------------------------------------------------
# 6. Entry point
# ---------------------------------------------------------------------------

def run_training_and_simulation(
    csv_path: str = "wtp_watertap_training_dataset.csv",
    results_csv_path: str = "wtp_simulation_results.csv",
):
    # Step 1: load and prepare data
    df = load_training_data(csv_path)

    # Step 2: train removal fraction r_tss using Pyomo (like video example)
    r_tss = train_tss_removal_fraction_with_pyomo(df)

    # Step 3: build minimal WaterTAP flowsheet with calibrated r_tss
    db = Database()
    m = build_dmf_flowsheet(db, r_tss)

    # Step 4: run WaterTAP across whole dataset
    results = simulate_dataset_with_dmf(m, df)

    # Step 5: save and plot
    results.to_csv(results_csv_path, index=False)
    print(f"Simulation results saved to {results_csv_path}")

    plot_results(results)

In [4]:
if __name__ == "__main__":
    run_training_and_simulation()

Calibrated TSS removal fraction (dataset-wide): 0.857


AttributeError: '_ScalarDualMediaFiltrationZO' object has no attribute 'inlet_state'

In [8]:
from idaes.core import FlowsheetBlock
from watertap.core.solvers import get_solver

from watertap.core.wt_database import Database
from watertap.core.zero_order_properties import WaterParameterBlock
from watertap.unit_models.zero_order import DualMediaFiltrationZO
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory


def main():

    # Create a Pyomo model and initialize the YAML database
    model = ConcreteModel()
    model.db = Database()

    # Create an IDAES flowsheet and define the solutes
    model.fs = FlowsheetBlock(dynamic=False)
    model.fs.params = WaterParameterBlock(solute_list=["nonvolatile_toc", "toc", "tss"])

    # Setup the zero-order model and define inlet flows
    model.fs.unit = DualMediaFiltrationZO(
        property_package=model.fs.params, database=model.db
    )
    model.fs.unit.inlet.flow_mass_comp[0, "H2O"].fix(10)
    model.fs.unit.inlet.flow_mass_comp[0, "nonvolatile_toc"].fix(1)
    model.fs.unit.inlet.flow_mass_comp[0, "toc"].fix(1)
    model.fs.unit.inlet.flow_mass_comp[0, "tss"].fix(1)

    # Load default parameters from the YAML database
    model.fs.unit.load_parameters_from_database()

    # Access the solver and solve the model
    solver = get_solver()
    solver.solve(model)

    # Display a report of the results
    model.fs.unit.report()

In [10]:
if __name__ == "__main__":
    main()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                              : Value      : Units         : Fixed : Bounds
                  Electricity Demand :     2373.6 :          watt : False : (0, None)
               Electricity Intensity : 1.8259e+05 :        pascal :  True : (None, None)
    Solute Removal [nonvolatile_toc] :    0.20000 : dimensionless :  True : (0, None)
                Solute Removal [toc] :    0.20000 : dimensionless :  True : (0, None)
                Solute Removal [tss] :    0.97000 : dimensionless :  True : (0, None)
                      Water Recovery :    0.99000 : dimensionless :  True : (0.0, 1.0000001)

------------------------------------------------------------------------------------
    Stream Table
                                               Units            Inlet   Treate